In [1]:
# load data
import dataloader
batch_size = 4
trainloader ,evalloader, testloader ,X_train ,X_test ,  y_train , y_test , inputFeatures, outputFeatures, datasetName, features_names = dataloader.load_kaggle_diabetes_dataset(batch_size=batch_size)
from sklearn.datasets import load_breast_cancer

print(features_names)
#data = load_breast_cancer()
#features_names = data.feature_names
#print(data)

/home/rosario/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


8
614
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


In [2]:
import train
import modelClass

model= modelClass.BinaryClassification(inputFeatures= inputFeatures, outputFeatures= outputFeatures)
num_epochs = 500
device = "cuda:0"
doEval = False
lr = 0.001

grads,grads_eval, grads_0, grads_epoch, grads_epoch_0, training_loss_batch, training_loss_epoch, training_acc, test_loss_batch, test_loss_epoch, test_acc= train.train_eval(trainloader,evalloader, testloader, model, num_epochs, device, y_train,y_test, lr, doEval= doEval)


[]
Epoch: 0
      Training_acc: 0.6644951140065146
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 1
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 2
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 3
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 4
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 5
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 6
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 7
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[]
Epoch: 8
      Training_acc: 0.6530944625407166
      Testing_acc : 0.7142857142857143
-------------------

[

In [3]:
import numpy as np
X_train = np.array(X_train)
print(len(features_names))
import lime 
import lime.lime_tabular

np.random.seed(1)
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, 
                                                   feature_names=features_names, 
                                                   class_names=['0', '1'])

print(explainer)

8


In [88]:
import utils
featureListAll  = utils.unpackingFeatureList(inputFeatures , grads)
np.max(featureListAll)
np.min(featureListAll)


-1.9373185634613037

In [99]:
from mlxtend.frequent_patterns import apriori, association_rules
import torch 
import numpy as np
from Myhelper_func import *
from tqdm import tqdm

import shap





predsList = []
with torch.no_grad():
        
        for inputs,labels in trainloader:
        
            inputs = inputs.to(device)
            labels = labels.to(device)  
    
            outputs = model(inputs)
            #print(outputs)
            __, preds = torch.max(outputs, 1)
            predsList.extend(preds)
print(len(predsList))  
X_train = np.array(X_train , dtype="float64")
df_X_train = pd.DataFrame(X_train, columns=list(features_names))
#print(df_X_train)
#JustData_df_X_train = df_X_train[1:]

#print(type(df_X_train.iloc[0]["mean radius"]))
from multiprocessing import Pool, cpu_count, Queue

num_cores = cpu_count()
print(f'num of cores: {num_cores}')
#print(len(X_train))
if len(intervals_dict) == 0:
        compute_intervals(intervals_dict, df_X_train, 1)
#print(len(intervals_dict))        


#itemset = set()

p = Pool(num_cores)


from utils import unpackingFeatureList

featureListAll  = unpackingFeatureList(inputFeatures , grads)
featureListAll = np.array(featureListAll)
featureListAll = featureListAll[:,-len(predsList):]

pos_label = '1'
neg_label = '0'


#print(itemset)
#print(len(itemset))


shap_threshold = 0.001
itemset = set()
encoded_vals = []
summed_values = {}
num_features = X_train.shape[1]

for feature in df_X_train.columns.to_list():
    if feature in intervals_dict:
        intervals = intervals_dict[feature]
        for interval in intervals:
            if interval != interval: continue
            left = interval.left
            right = interval.right
            name = f'{left}<{feature}<={right}'
            itemset.add(name)
    else:
        itemset.add(feature)
        
itemset.add(pos_label)
itemset.add(neg_label)

for indx in tqdm(range(len(predsList))):
    # since we have two inputs we pass a list of inputs to the explainer
    #explainer = shap.GradientExplainer(model, X_train)

    # we explain the model's predictions on the first three samples of the test set
    #shap_values = explainer.shap_values([X_test, X_test])
    pos_queue.put(pos_label)
    neg_queue.put(neg_label)
    

    exp = [item[indx] for item in featureListAll] #normalize featureListALL ?
    instance_features = df_X_train.iloc[[indx]].to_dict(orient='records')[0]

    feature_vals = [instance_features[name] for name in features_names] #put here grads# feature values ?? 

    zipped = zip(exp, feature_vals,
                 features_names, [shap_threshold]*len(features_names))

    p.map(get_relevant_features, zipped) # if statisfies threshold get one hot encoded one lese zero
    append_to_encoded_vals(pos_queue, itemset, encoded_vals) #
    append_to_encoded_vals(neg_queue, itemset, encoded_vals) #


ohe_df = pd.DataFrame(encoded_vals)

#
# print(ohe_df)


614
num of cores: 12


100%|██████████| 614/614 [00:01<00:00, 539.54it/s]


In [100]:
#from Myhelper_func import *
#shaplyValue = calculateShaplyValue(model,X_train[0],device, X_train,0,0)


In [101]:
ohe_df

,0.0329<Age<=1.0,0,-0.001<Glucose<=1.0,-0.000171<Pregnancies<=0.171,-0.000596<BMI<=0.596,1,-0.0006<SkinThickness<=0.6,-0.001<Insulin<=1.0,-0.001<BloodPressure<=1.0,-1.85e-05<DiabetesPedigreeFunction<=0.0185
0,0,0,0,0,1,1,1,1,1,1
1,1,1,1,1,0,0,0,0,0,0
2,0,0,0,0,1,1,1,0,1,1
3,1,1,1,1,0,0,0,1,0,0
4,0,0,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
1223,1,1,1,1,0,0,1,1,0,0
1224,1,0,0,0,1,1,0,1,1,1
1225,0,1,1,1,0,0,1,0,0,0
1226,0,0,0,0,1,1,1,0,1,1


In [102]:
     
#Get frequent itemsets from a one-hot DataFrame
                                        #10
print(10/len(predsList))
freq_items = apriori(ohe_df, min_support=(10/len(predsList)), use_colnames=True, max_len=3)
print(freq_items)
all_rules = association_rules(freq_items, metric="confidence", min_threshold=0.7, support_only=False)
                                                                    #10
freq_items = apriori(ohe_df.loc[ohe_df[pos_label] == 1], min_support=(10/len(predsList)), use_colnames=True, max_len=3)
pos_rules = association_rules(freq_items, metric="confidence", min_threshold=0.6, support_only=False)
                                                            #10
freq_items = apriori(ohe_df.loc[ohe_df[neg_label] == 1], min_support=(10/len(predsList)), use_colnames=True, max_len=3)
neg_rules = association_rules(freq_items, metric="confidence", min_threshold=0.6, support_only=False)

0.016286644951140065
      support                                           itemsets
0    0.496743                                  (0.0329<Age<=1.0)
1    0.500000                                                (0)
2    0.500000                              (-0.001<Glucose<=1.0)
3    0.499186                     (-0.000171<Pregnancies<=0.171)
4    0.499186                             (-0.000596<BMI<=0.596)
..        ...                                                ...
150  0.206840  (-1.85e-05<DiabetesPedigreeFunction<=0.0185, 1...
151  0.229642  (-0.001<Insulin<=1.0, -0.001<BloodPressure<=1....
152  0.122964  (-1.85e-05<DiabetesPedigreeFunction<=0.0185, -...
153  0.152280  (-1.85e-05<DiabetesPedigreeFunction<=0.0185, -...
154  0.121336  (-1.85e-05<DiabetesPedigreeFunction<=0.0185, -...

[155 rows x 2 columns]


DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type


In [103]:
positive = all_rules[all_rules['consequents'] == {pos_label}]
positive = positive[positive['confidence'] == 1]
positive = positive.sort_values(['confidence', 'support'], ascending=[False, False])

seen = set()
dropped = set()
indexes_to_drop = []

positive = positive.reset_index(drop=True)
print(len(positive))
for i in positive.index:
    new_rule = positive.loc[[i]]['antecedents'].values[0]
    
    for seen_rule in seen:
        if seen_rule.issubset(new_rule):#new_rule.issubset(seen_rule) or seen_rule.issubset(new_rule):
            indexes_to_drop.append(i)
            break
    else:
        seen.add(new_rule)

positive.drop(positive.index[indexes_to_drop], inplace=True )
print(len(positive))
positive

0
0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [104]:
negative = all_rules[all_rules['consequents'] == {neg_label}]
negative = negative[negative['confidence'] == 1]

negative = negative.sort_values(['confidence', 'support'], ascending=[False, False])

seen = set()
dropped = set()
indexes_to_drop = []

negative = negative.reset_index(drop=True)
print(len(negative))
for i in negative.index:
    new_rule = negative.loc[[i]]['antecedents'].values[0]
    
    for seen_rule in seen:
        if seen_rule.issubset(new_rule):#new_rule.issubset(seen_rule) or seen_rule.issubset(new_rule):
            indexes_to_drop.append(i)
            break
    else:
        seen.add(new_rule)

negative.drop(negative.index[indexes_to_drop], inplace=True )
print(len(negative))
negative

0
0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [105]:
positive['num-items'] = positive['antecedents'].map(lambda x: len(x))
negative['num-items'] = negative['antecedents'].map(lambda x: len(x))
positive['consequents'] = positive['consequents'].map(lambda x: pos_label)
negative['consequents'] = negative['consequents'].map(lambda x: neg_label)

both = positive.append(negative, ignore_index=True)

discr_rules = both[['antecedents', 'consequents', 'num-items', 'support', 'confidence', 'antecedent support']].sort_values(
    ['support', 'confidence', 'num-items'], ascending=[False, False, False])

discr_rules = discr_rules.rename(columns={"antecedents": "itemset", "consequents": "label"})

discr_rules

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,itemset,label,num-items,support,confidence,antecedent support


In [106]:
rev_positive = pos_rules[pos_rules['antecedents'] == {pos_label}]
rev_positive = rev_positive[rev_positive['confidence'] >= 0.7]
rev_positive = rev_positive.sort_values(['confidence', 'support'], ascending=[False, False])

seen = set()
dropped = set()
indexes_to_drop = []

rev_positive = rev_positive.reset_index(drop=True)
print(len(rev_positive))
for i in rev_positive.index:
    new_rule = rev_positive.loc[[i]]['consequents'].values[0]
    
    for seen_rule, indx in seen:
        if seen_rule.issubset(new_rule):
            indexes_to_drop.append(i)
            break
    else:
        seen.add((new_rule, i))

rev_positive.drop(rev_positive.index[indexes_to_drop], inplace=True )
print(len(rev_positive))



rev_negative = neg_rules[neg_rules['antecedents'] == {neg_label}]
rev_negative = rev_negative[rev_negative['confidence'] >= 0.7]
rev_negative = rev_negative.sort_values(['confidence', 'support'], ascending=[False, False])

seen = set()
dropped = set()
indexes_to_drop = []

rev_negative = rev_negative.reset_index(drop=True)
print(len(rev_negative))
for i in rev_negative.index:
    new_rule = rev_negative.loc[[i]]['consequents'].values[0]
    
    for seen_rule, indx in seen:
        if seen_rule.issubset(new_rule):
            indexes_to_drop.append(i)
            break
    else:
        seen.add((new_rule, i))

rev_negative.drop(rev_negative.index[indexes_to_drop], inplace=True )
print(len(rev_negative))

3
2
1
1


In [107]:
rev_positive['num-items'] = rev_positive['consequents'].map(lambda x: len(x))
rev_negative['num-items'] = rev_negative['consequents'].map(lambda x: len(x))
rev_positive['antecedents'] = rev_positive['antecedents'].map(lambda x: pos_label)
rev_negative['antecedents'] = rev_negative['antecedents'].map(lambda x: neg_label)

rev_both = rev_positive.append(rev_negative, ignore_index=True)

chr_rules = rev_both[['antecedents', 'consequents', 'num-items', 'support', 
                          'confidence', 'consequent support']].sort_values(
    ['support', 'confidence', 'num-items'], ascending=[False, False, False])

chr_rules = chr_rules.rename(columns={"antecedents": "label", "consequents": "itemset"})

chr_rules

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,label,itemset,num-items,support,confidence,consequent support
0,1,(-0.000596<BMI<=0.596),1,0.817590,0.817590,0.817590
2,0,(-0.001<Glucose<=1.0),1,0.793160,0.793160,0.793160
1,1,(-0.001<BloodPressure<=1.0),1,0.762215,0.762215,0.762215


In [108]:
from rules_model import *

dev_pred = xgb_clf.predict(X_dev)
rules_model = RulesModel(ohe_df, discr_rules, y_dev, pos_label, neg_label)

test_pred = xgb_clf.predict(X_test)
rules_test_sol = rules_model.eval_rules(X_test, test_pred, alpha=10, beta =1, decision_thr=0.97)

NameError: name 'xgb_clf' is not defined